In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("Claim Amount Data Set").setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.sql.functions import isnan, when, count, col, countDistinct

In [4]:
DataSchema = StructType([
    StructField("Row_ID", IntegerType(), True),
    StructField("Household_ID", IntegerType(), True),
    StructField("Vehicle", IntegerType(), True),
    StructField("Calendar_Year", IntegerType(), True),
    StructField("Model_Year", IntegerType(), True),
    StructField("Blind_Make", StringType(), True),
    StructField("Blind_Model", StringType(), True),
    StructField("Blind_Submodel", StringType(), True),        
    StructField("Cat1", StringType(), True),
    StructField("Cat2", StringType(), True),
    StructField("Cat3", StringType(), True),
    StructField("Cat4", StringType(), True),
    StructField("Cat5", StringType(), True),
    StructField("Cat6", StringType(), True),
    StructField("Cat7", StringType(), True),
    StructField("Cat8", StringType(), True),
    StructField("Cat9", StringType(), True),
    StructField("Cat10", StringType(), True),
    StructField("Cat11", StringType(), True),
    StructField("Cat12", StringType(), True),
    StructField("OrdCat", IntegerType(), True),
    StructField("Var1", DoubleType(), True),
    StructField("Var2", DoubleType(), True),
    StructField("Var3", DoubleType(), True),
    StructField("Var4", DoubleType(), True),
    StructField("Var5", DoubleType(), True),
    StructField("Var6", DoubleType(), True),
    StructField("Var7", DoubleType(), True),
    StructField("Var8", DoubleType(), True),
    StructField("NVCat", StringType(), True),
    StructField("NVVar1", DoubleType(), True),
    StructField("NVVar2", DoubleType(), True),
    StructField("NVVar3", DoubleType(), True),
    StructField("NVVar4", DoubleType(), True),
    StructField("Claim_Amount", DoubleType(), True)])

#Q1:Create a dataframe for above csv data.using spark.read.format()
(First line in the dataset is header).

In [5]:
## Read data and create a dataframe
data = spark.read.format("csv")\
       .option("header", "true")\
       .option("inferSchema", "true")\
       .option("nullValue", "?")\
       .option("ignoreTrailingWhiteSpace","true")\
       .option("ignoreLeadingWhiteSpace","true")\
       .option("quote",'"')\
       .load('file:///home/1844B39/CUTe5/sample_set.csv', schema = DataSchema)

#data=spark.read.csv("file:///home/1844B39/CUTe5/sample_set.csv",nullValue="?",nanValue="NA",header=True,ignoreTrailingWhiteSpace=True,ignoreLeadingWhiteSpace=True,inferSchema=True)

##Understanding the data

In [6]:
data.printSchema()

root
 |-- Row_ID: integer (nullable = true)
 |-- Household_ID: integer (nullable = true)
 |-- Vehicle: integer (nullable = true)
 |-- Calendar_Year: integer (nullable = true)
 |-- Model_Year: integer (nullable = true)
 |-- Blind_Make: string (nullable = true)
 |-- Blind_Model: string (nullable = true)
 |-- Blind_Submodel: string (nullable = true)
 |-- Cat1: string (nullable = true)
 |-- Cat2: string (nullable = true)
 |-- Cat3: string (nullable = true)
 |-- Cat4: string (nullable = true)
 |-- Cat5: string (nullable = true)
 |-- Cat6: string (nullable = true)
 |-- Cat7: string (nullable = true)
 |-- Cat8: string (nullable = true)
 |-- Cat9: string (nullable = true)
 |-- Cat10: string (nullable = true)
 |-- Cat11: string (nullable = true)
 |-- Cat12: string (nullable = true)
 |-- OrdCat: integer (nullable = true)
 |-- Var1: double (nullable = true)
 |-- Var2: double (nullable = true)
 |-- Var3: double (nullable = true)
 |-- Var4: double (nullable = true)
 |-- Var5: double (nullable = tru

In [7]:
data.dtypes

[('Row_ID', 'int'),
 ('Household_ID', 'int'),
 ('Vehicle', 'int'),
 ('Calendar_Year', 'int'),
 ('Model_Year', 'int'),
 ('Blind_Make', 'string'),
 ('Blind_Model', 'string'),
 ('Blind_Submodel', 'string'),
 ('Cat1', 'string'),
 ('Cat2', 'string'),
 ('Cat3', 'string'),
 ('Cat4', 'string'),
 ('Cat5', 'string'),
 ('Cat6', 'string'),
 ('Cat7', 'string'),
 ('Cat8', 'string'),
 ('Cat9', 'string'),
 ('Cat10', 'string'),
 ('Cat11', 'string'),
 ('Cat12', 'string'),
 ('OrdCat', 'int'),
 ('Var1', 'double'),
 ('Var2', 'double'),
 ('Var3', 'double'),
 ('Var4', 'double'),
 ('Var5', 'double'),
 ('Var6', 'double'),
 ('Var7', 'double'),
 ('Var8', 'double'),
 ('NVCat', 'string'),
 ('NVVar1', 'double'),
 ('NVVar2', 'double'),
 ('NVVar3', 'double'),
 ('NVVar4', 'double'),
 ('Claim_Amount', 'double')]

##Total no. of Rows and Columns

In [8]:
print("No. of Columns = {}".format(len(data.columns)))

print('No. of Records = {}'.format(data.count()))

No. of Columns = 35
No. of Records = 1000000


In [9]:
data.head(3)

[Row(Row_ID=11133137, Household_ID=6690958, Vehicle=1, Calendar_Year=2006, Model_Year=1992, Blind_Make=u'BW', Blind_Model=u'BW.33', Blind_Submodel=u'BW.33.1', Cat1=u'D', Cat2=None, Cat3=u'C', Cat4=None, Cat5=None, Cat6=u'E', Cat7=None, Cat8=u'A', Cat9=u'B', Cat10=u'A', Cat11=u'B', Cat12=u'D', OrdCat=5, Var1=3.4962213, Var2=0.740051, Var3=1.0972409, Var4=1.0774881, Var5=2.7216895, Var6=2.4425492, Var7=1.6081873, Var8=-0.1821408, NVCat=u'O', NVVar1=-0.2315299, NVVar2=-0.2661168, NVVar3=-0.2723372, NVVar4=-0.2514189, Claim_Amount=0.0),
 Row(Row_ID=12066135, Household_ID=7022371, Vehicle=2, Calendar_Year=2005, Model_Year=1995, Blind_Make=u'M', Blind_Model=u'M.16', Blind_Submodel=u'M.16.1', Cat1=u'B', Cat2=None, Cat3=u'A', Cat4=u'A', Cat5=u'A', Cat6=u'C', Cat7=u'C', Cat8=u'A', Cat9=u'B', Cat10=u'B', Cat11=u'A', Cat12=u'D', OrdCat=2, Var1=-0.7703518, Var2=-1.5077976, Var3=-1.0316697, Var4=-1.5441353, Var5=-0.8822796, Var6=-1.4771497, Var7=-1.2548265, Var8=-0.9952642, NVCat=u'M', NVVar1=-0.23

In [10]:
data.show(3)

+--------+------------+-------+-------------+----------+----------+-----------+--------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+------+----------+----------+----------+----------+----------+----------+----------+----------+-----+----------+----------+----------+----------+------------+
|  Row_ID|Household_ID|Vehicle|Calendar_Year|Model_Year|Blind_Make|Blind_Model|Blind_Submodel|Cat1|Cat2|Cat3|Cat4|Cat5|Cat6|Cat7|Cat8|Cat9|Cat10|Cat11|Cat12|OrdCat|      Var1|      Var2|      Var3|      Var4|      Var5|      Var6|      Var7|      Var8|NVCat|    NVVar1|    NVVar2|    NVVar3|    NVVar4|Claim_Amount|
+--------+------------+-------+-------------+----------+----------+-----------+--------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+------+----------+----------+----------+----------+----------+----------+----------+----------+-----+----------+----------+----------+----------+------------+
|11133137|     6690958|      1|         2006|      1

## Q2:Verify summary of the dataframe (how many rows and columns)

In [11]:
data.describe().show()

+-------+------------------+------------------+------------------+-----------------+-----------------+----------+-----------+--------------+------+------+------+------+------+------+------+------+-------+------+------+------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------+--------------------+-------------------+-------------------+--------------------+------------------+
|summary|            Row_ID|      Household_ID|           Vehicle|    Calendar_Year|       Model_Year|Blind_Make|Blind_Model|Blind_Submodel|  Cat1|  Cat2|  Cat3|  Cat4|  Cat5|  Cat6|  Cat7|  Cat8|   Cat9| Cat10| Cat11| Cat12|            OrdCat|                Var1|                Var2|                Var3|                Var4|                Var5|                Var6|               Var7|                Var8|  NVCat|              NVVar1|             NVVar2|             NVVar3

##Q3:As the '?' describes the unknown data in each column, identify
##how many columns exists with '?' (NAs)

In [12]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+------+------------+-------+-------------+----------+----------+-----------+--------------+----+------+----+------+------+----+------+----+----+-----+-----+-----+------+----+----+----+----+----+----+----+----+-----+------+------+------+------+------------+
|Row_ID|Household_ID|Vehicle|Calendar_Year|Model_Year|Blind_Make|Blind_Model|Blind_Submodel|Cat1|  Cat2|Cat3|  Cat4|  Cat5|Cat6|  Cat7|Cat8|Cat9|Cat10|Cat11|Cat12|OrdCat|Var1|Var2|Var3|Var4|Var5|Var6|Var7|Var8|NVCat|NVVar1|NVVar2|NVVar3|NVVar4|Claim_Amount|
+------+------------+-------+-------------+----------+----------+-----------+--------------+----+------+----+------+------+----+------+----+----+-----+-----+-----+------+----+----+----+----+----+----+----+----+-----+------+------+------+------+------------+
|     0|           0|      0|            0|         0|       653|        653|           653|1995|369621| 325|426569|426969|1995|543322| 278|   0|  314| 2316| 2134|   568|   0|   0|   0|   0|   0|   0|   0|   0|    0|     0|   

##Q5:Remove all the columns where the no. of rows with '?' exceeds 35%
#for that column.

In [13]:
data=data.drop('Cat2','Cat4','Cat5','Cat7')

In [14]:
data.show(2)

+--------+------------+-------+-------------+----------+----------+-----------+--------------+----+----+----+----+----+-----+-----+-----+------+----------+----------+----------+----------+----------+----------+----------+----------+-----+----------+----------+----------+----------+------------+
|  Row_ID|Household_ID|Vehicle|Calendar_Year|Model_Year|Blind_Make|Blind_Model|Blind_Submodel|Cat1|Cat3|Cat6|Cat8|Cat9|Cat10|Cat11|Cat12|OrdCat|      Var1|      Var2|      Var3|      Var4|      Var5|      Var6|      Var7|      Var8|NVCat|    NVVar1|    NVVar2|    NVVar3|    NVVar4|Claim_Amount|
+--------+------------+-------+-------------+----------+----------+-----------+--------------+----+----+----+----+----+-----+-----+-----+------+----------+----------+----------+----------+----------+----------+----------+----------+-----+----------+----------+----------+----------+------------+
|11133137|     6690958|      1|         2006|      1992|        BW|      BW.33|       BW.33.1|   D|   C|   E|   

In [15]:
data=data.drop('OrdCat')

##Q4:Replace all '?' values in the result data frame as NAs.

#Done while reading the csv

In [16]:
print("No. of Columns = {}".format(len(data.columns)))

No. of Columns = 30


#Q6:Fill remaining Null Values with Zeroes in the entire dataset, if any.

In [17]:
data= data.select(data.columns[0:31]).fillna("0")

In [18]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+------+------------+-------+-------------+----------+----------+-----------+--------------+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+-----+------+------+------+------+------------+
|Row_ID|Household_ID|Vehicle|Calendar_Year|Model_Year|Blind_Make|Blind_Model|Blind_Submodel|Cat1|Cat3|Cat6|Cat8|Cat9|Cat10|Cat11|Cat12|Var1|Var2|Var3|Var4|Var5|Var6|Var7|Var8|NVCat|NVVar1|NVVar2|NVVar3|NVVar4|Claim_Amount|
+------+------------+-------+-------------+----------+----------+-----------+--------------+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+----+-----+------+------+------+------+------------+
|     0|           0|      0|            0|         0|         0|          0|             0|   0|   0|   0|   0|   0|    0|    0|    0|   0|   0|   0|   0|   0|   0|   0|   0|    0|     0|     0|     0|     0|           0|
+------+------------+-------+-------------+----------+----------+-----------+--------------+----+----+----+-

##Q7:Derive a new column Vehicle_age = Current Year - Model year

In [19]:
data=data.withColumn('Vehicle_age', data.Calendar_Year - data.Model_Year)

In [20]:
data.select("Calendar_Year","Model_Year","Vehicle_age").show(10)

+-------------+----------+-----------+
|Calendar_Year|Model_Year|Vehicle_age|
+-------------+----------+-----------+
|         2006|      1992|         14|
|         2005|      1995|         10|
|         2007|      2000|          7|
|         2005|      1992|         13|
|         2006|      2003|          3|
|         2005|      2005|          0|
|         2007|      1995|         12|
|         2007|      2000|          7|
|         2006|      1998|          8|
|         2006|      2000|          6|
+-------------+----------+-----------+
only showing top 10 rows



#Q8:Extract all the non-zero records from the dataset(> 0) into a new
#dataframe and verify the number of rows.

In [21]:
data2=data.where(data.Claim_Amount > 0)

In [22]:
data2.select('Claim_Amount').count()

7249

#Q9:Remove the columns Row_ID, Household_ID and Vehicle from the
#original dataframe.

In [23]:
data2=data2.drop('Household_ID','Vehicle','Row_ID')

#Q10: Create two new dataframes
A. train_DF, For the data where Calendar_Year = 2005 and 2006
B. test_DF, For the data where Calendar_Year = 2007

In [24]:
train_DF=data2.where((data2.Calendar_Year == 2005 )| (data2.Calendar_Year == 2006))

In [25]:
test_DF=data2.where(data2.Calendar_Year == 2007)

In [26]:
data2.dtypes

[('Calendar_Year', 'int'),
 ('Model_Year', 'int'),
 ('Blind_Make', 'string'),
 ('Blind_Model', 'string'),
 ('Blind_Submodel', 'string'),
 ('Cat1', 'string'),
 ('Cat3', 'string'),
 ('Cat6', 'string'),
 ('Cat8', 'string'),
 ('Cat9', 'string'),
 ('Cat10', 'string'),
 ('Cat11', 'string'),
 ('Cat12', 'string'),
 ('Var1', 'double'),
 ('Var2', 'double'),
 ('Var3', 'double'),
 ('Var4', 'double'),
 ('Var5', 'double'),
 ('Var6', 'double'),
 ('Var7', 'double'),
 ('Var8', 'double'),
 ('NVCat', 'string'),
 ('NVVar1', 'double'),
 ('NVVar2', 'double'),
 ('NVVar3', 'double'),
 ('NVVar4', 'double'),
 ('Claim_Amount', 'double'),
 ('Vehicle_age', 'int')]

#Q11:Separate into Categorical and Continuous attributes.

In [27]:
cat_Var_Names = ['Cat1','Cat3','Cat6','Cat8','Cat9','Cat10','Cat11','Cat12','NVCat']

num_Var_Names = ['Calendar_Year','Model_Year','Var1','Var2','Var3','Var4','Var5','Var6','Var7','Var8','NVVar1','NVVar2','NVVar3','NVVar4','Vehicle_age']

In [28]:
from pyspark.ml.feature import VectorAssembler

assembler_Num = VectorAssembler(inputCols=num_Var_Names,outputCol="num_features")

In [29]:
from pyspark.ml.feature import MinMaxScaler

min_Max_Scalar = MinMaxScaler(inputCol="num_features",outputCol="scaled_num_features")

#Q12:Encode Categorical attributes. (Dummy)

In [30]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

indexers_Cat = [StringIndexer(inputCol=cat_Var_Name, outputCol="{0}_index".format(cat_Var_Name)) for cat_Var_Name in cat_Var_Names ]
encoders_Cat = [OneHotEncoder(inputCol=indexer.getOutputCol(),outputCol="{0}_vec".format(indexer.getInputCol())) for indexer in indexers_Cat]
assembler_Cat = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders_Cat],outputCol="cat_features")

assembler = VectorAssembler(inputCols=["scaled_num_features", "cat_features"], outputCol="features")

#Q13:Create a vector for all the predictor variable.

In [31]:
preprocessiong_Stages = [assembler_Num]+[min_Max_Scalar]+indexers_Cat+encoders_Cat+[assembler_Cat]+[assembler]

#Q14:Set the target variable (label)

In [32]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter=10,labelCol="Claim_Amount",featuresCol="features")

In [33]:
from pyspark.ml import Pipeline

lr_Pipeline = Pipeline(stages=preprocessiong_Stages+[lr])

lr_Pipeline_model = lr_Pipeline.fit(train_DF)

In [34]:
print("Coefficients: " + str(lr_Pipeline_model.stages[-1].coefficients))
print("Intercept: " + str(lr_Pipeline_model.stages[-1].intercept))

Coefficients: [-16.6331716686,-17.7237690177,-156.091775741,-43.2731039348,205.762211142,-25.2403535726,10.5325356314,-26.2854096682,-38.5624478296,-116.930534108,1.97391779298,-60.4440147853,-127.13425807,-75.9438903337,12.5478469249,-5.81290224198,21.1468045337,39.4230103955,8.68020590486,-45.1281314465,-15.4075147535,-12.4507001397,-10.8016171057,-73.3680026176,-100.053398712,-10.8318346916,47.8316666295,-36.5846492835,3.23314450931,-12.3632175203,1.07887142713,-12.144508321,-1.90588693731,33.1490955793,-14.8120088349,-12.7884425866,3.71686549559,1.70936095573,-8.97185197674,-8.02194151158,2.55990747788,-0.388889873628,8.62385280362,5.00976740969,-5.19664444067,-17.53184659,22.0157355399,13.7357194818,-20.5326401431,-0.898932825101,-1.41458364105,-3.40253724119,39.3054311726,-57.781159864,32.9039800481,6.00326412545,-30.2010165692,-20.1591791388,24.1557461019,-11.5485701054,22.3977302263,-65.4700410801,-23.4435645212,-72.2255934622,-132.581486846,-92.6773870882,-30.4807924913,-29.34

In [35]:
lr_Summary = lr_Pipeline_model.stages[-1].summary
print(lr_Summary)

In [36]:
train_predictions_lr = lr_Pipeline_model.transform(train_DF)
test_predictions_lr = lr_Pipeline_model.transform(test_DF)

In [37]:
test_predictions_lr.dtypes

[('Calendar_Year', 'int'),
 ('Model_Year', 'int'),
 ('Blind_Make', 'string'),
 ('Blind_Model', 'string'),
 ('Blind_Submodel', 'string'),
 ('Cat1', 'string'),
 ('Cat3', 'string'),
 ('Cat6', 'string'),
 ('Cat8', 'string'),
 ('Cat9', 'string'),
 ('Cat10', 'string'),
 ('Cat11', 'string'),
 ('Cat12', 'string'),
 ('Var1', 'double'),
 ('Var2', 'double'),
 ('Var3', 'double'),
 ('Var4', 'double'),
 ('Var5', 'double'),
 ('Var6', 'double'),
 ('Var7', 'double'),
 ('Var8', 'double'),
 ('NVCat', 'string'),
 ('NVVar1', 'double'),
 ('NVVar2', 'double'),
 ('NVVar3', 'double'),
 ('NVVar4', 'double'),
 ('Claim_Amount', 'double'),
 ('Vehicle_age', 'int'),
 ('num_features', 'vector'),
 ('scaled_num_features', 'vector'),
 ('Cat1_index', 'double'),
 ('Cat3_index', 'double'),
 ('Cat6_index', 'double'),
 ('Cat8_index', 'double'),
 ('Cat9_index', 'double'),
 ('Cat10_index', 'double'),
 ('Cat11_index', 'double'),
 ('Cat12_index', 'double'),
 ('NVCat_index', 'double'),
 ('Cat1_vec', 'vector'),
 ('Cat3_vec', 'vect

In [38]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="Claim_Amount",predictionCol="prediction",metricName="rmse")

predictionAndLabels_train_lr = train_predictions_lr.select("prediction", "Claim_Amount")
train_rsquared_lr = evaluator.evaluate(predictionAndLabels_train_lr)

print("RMSE  = " + str(train_rsquared_lr))

predictionAndLabels_test_lr = test_predictions_lr.select("prediction", "Claim_Amount")
test_rsquared_lr = evaluator.evaluate(predictionAndLabels_test_lr)


print("RMSE = " + str(test_rsquared_lr))

RMSE  = 451.803129759
RMSE = 312.574174418


In [39]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [44]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1]) \
    .addGrid(lr.elasticNetParam, [0.5])\
    .addGrid(lr.maxIter,[100])\
    .build()
    
lr_crossval = CrossValidator(estimator=lr_Pipeline,
                             estimatorParamMaps=paramGrid,
                             evaluator=evaluator,
                             numFolds=2)     

In [48]:
#Run cross-validation, and choose the best set of parameters.
lr_crossval_Model = lr_crossval.fit(train_DF)

In [ ]:
train_predictions_lrcv = lr_crossval_Model.transform(train_DF)
test_predictions_lrcv = lr_crossval_Model.transform(test_DF)

In [ ]:
predictionAndLabels_train_lrcv = train_predictions_lrcv.select("prediction", "Claim_Amount")
train_rmsecv = evaluator.evaluate(predictionAndLabels_train_lrcv)
print("RMSE  = " + str(train_rmsecv))

predictionAndLabels_test_lrcv = test_predictions_lrcv.select("prediction", "Claim_Amount")
test_rmsecv = evaluator.evaluate(predictionAndLabels_test_lrcv)
print("RMSE = " + str(test_rmsecv))

In [46]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(labelCol="Claim_Amount", featuresCol="features")

In [47]:
dt_Pipeline = Pipeline(stages=preprocessiong_Stages+[dt]) 

dt_Pipeline_model = dt_Pipeline.fit(train_DF)

In [49]:
train_predictions_dt = dt_Pipeline_model.transform(train_DF)
test_predictions_dt = dt_Pipeline_model.transform(test_DF)

In [50]:
test_predictions_dt.show(2)

+-------------+----------+----------+-----------+--------------+----+----+----+----+----+-----+-----+-----+----------+----------+----------+----------+----------+----------+---------+----------+-----+----------+----------+----------+----------+------------+-----------+--------------------+--------------------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+--------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------+--------------------+--------------------+------------------+
|Calendar_Year|Model_Year|Blind_Make|Blind_Model|Blind_Submodel|Cat1|Cat3|Cat6|Cat8|Cat9|Cat10|Cat11|Cat12|      Var1|      Var2|      Var3|      Var4|      Var5|      Var6|     Var7|      Var8|NVCat|    NVVar1|    NVVar2|    NVVar3|    NVVar4|Claim_Amount|Vehicle_age|        num_features| scaled_num_features|Cat1_index|Cat3_index|Cat6_index|Cat8_index|Cat9_index|Cat10_index|Cat11_index|Cat12_

In [51]:
predictionAndLabels_train_dt = train_predictions_dt.select("prediction", "Claim_Amount")
train_rmse_dt = evaluator.evaluate(predictionAndLabels_train_dt)

print("RMSE  = " + str(train_rmse_dt))

predictionAndLabels_test_dt = test_predictions_dt.select("prediction", "Claim_Amount")
test_rmse_dt = evaluator.evaluate(predictionAndLabels_test_dt)

print("RMSE = " + str(test_rmse_dt))

RMSE  = 438.563460983
RMSE = 323.986697624


In [52]:
paramGridDT = ParamGridBuilder()\
    .addGrid(dt.maxDepth, [4]) \
    .build()
    
dt_crossval = CrossValidator(estimator=dt_Pipeline,
                             estimatorParamMaps=paramGridDT,
                             evaluator=evaluator,
                             numFolds=2)     

In [54]:
# Run cross-validation, and choose the best set of parameters.
dt_crossval_Model = dt_crossval.fit(train_DF)

In [ ]:
train_predictions_dtcv = dt_crossval_Model.transform(train_DF)
test_predictions_dtcv = dt_crossval_Model.transform(test_DF)

In [ ]:
predictionAndLabels_train_dtcv = train_predictions_dtcv.select("prediction", "Claim_Amount")
train_rmsedtcv = evaluator.evaluate(predictionAndLabels_train_dtcv)
print("RMSE  = " + str(train_rmsedtcv))

predictionAndLabels_test_dtcv = test_predictions_dtcv.select("prediction", "Claim_Amount")
test_rmsedtcv = evaluator.evaluate(predictionAndLabels_test_dtcv)
print("RMSE = " + str(test_rmsedtcv))